Render

In [1]:
import importlib
import os
from tkinter import NO
import yaml
import argparse
from datetime import datetime

from sacd.env import make_pytorch_env
from sacd.agent import SacdAgent, SharedSacdAgent


import torch
import torch.nn as nn
from torch.distributions import MultivariateNormal
from torch.distributions import Categorical
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
# import gymnasium as gym
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT,COMPLEX_MOVEMENT
from datetime import datetime
import cv2
from tqdm import tqdm
import numpy as np
from datetime import datetime
from gym.wrappers import GrayScaleObservation
from smb_env_fct import renderEnv, plotRewardandTime, DeadlockEnv, SkipFrame, Downsample, GrayScale, move_state_channels, MarioDeathLoggerWrapper, RandomEnvWrapper


/home/gong/miniconda3/envs/sacd/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
parser = argparse.ArgumentParser()
parser.add_argument(
    '--config', type=str, default=os.path.join('config', 'sacd.yaml'))
parser.add_argument('--shared', action='store_true')
parser.add_argument('--env_id', type=str, default='SuperMarioBros-v2')
parser.add_argument('--log_folder', type=str, default='logs')
parser.add_argument('--cuda', action='store_true')
parser.add_argument('--seed', type=int, default=0)
args, unknown = parser.parse_known_args()

with open(args.config) as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)

In [3]:
frameskip = 4
# load_model_path = "model/SuperMarioBrosRandomStages-v2"
load_model_path = "model/SuperMarioBros-v2"


# Specify the directory to log.
name = args.config.split('/')[-1].rstrip('.yaml')
if args.shared:
    name = 'shared-' + name
time = datetime.now().strftime("%Y%m%d-%H%M")
log_dir = os.path.join(
    args.log_folder, args.env_id, f'{name}-seed{args.seed}-{time}')

logfile = f"{log_dir}/mario_death_logfile_{time}.txt"

In [4]:
# env = gym_super_mario_bros.make(args.env_id, apply_api_compatibility=True)  #the environment. v0 is with original background, v1 has the background removed
# env = gym_super_mario_bros.make("SuperMarioBros-1-1-v2", apply_api_compatibility=True)
stages = ['1-1', '1-2', '1-3']
env = RandomEnvWrapper(args.env_id, stages)
env = JoypadSpace(env, SIMPLE_MOVEMENT)               #The Joypadspace sets the available actions. We use SIMPLE_MOVEMENT.
env = SkipFrame(env, skip=frameskip)                  #Skipframewrapper to skip some frames
env = DeadlockEnv(env,threshold=10)                   #Deadlock environment wrapper to stop the game if mario is stuck at a pipe
env = MarioDeathLoggerWrapper(env, logfile=logfile, env_id=None, select_random_stage=True)

/home/gong/miniconda3/envs/sacd/lib/python3.9/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment SuperMarioBros-1-1-v2 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
/home/gong/miniconda3/envs/sacd/lib/python3.9/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment SuperMarioBros-1-2-v2 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
/home/gong/miniconda3/envs/sacd/lib/python3.9/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment SuperMarioBros-1-3-v2 is out of date. You should consider upgrading to version `v3`.
  logger.warn(


In [5]:
# Specify the directory to log.
name = args.config.split('/')[-1].rstrip('.yaml')
if args.shared:
    name = 'shared-' + name
time = datetime.now().strftime("%Y%m%d-%H%M")
log_dir = os.path.join(
    'logs', args.env_id, f'{name}-seed{args.seed}-{time}')

# Create the agent.
Agent = SacdAgent if not args.shared else SharedSacdAgent
agent = Agent(
    env=env, test_env=env, log_dir=log_dir, cuda=args.cuda,
    seed=args.seed, **config)

In [6]:
# agent.evaluate()
agent.load_models(load_model_path)
agent.testnshow()
del agent

/home/gong/miniconda3/envs/sacd/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


mean return: 96.0 


This browser does not support the video tag.


Exception ignored in: <function BaseAgent.__del__ at 0x7f4f5a4cf1f0>
Traceback (most recent call last):
  File "/home/gong/Projects/sac-discrete-pytorch/sacd/agent/base.py", line 283, in __del__
    self.test_env.close()
  File "/home/gong/miniconda3/envs/sacd/lib/python3.9/site-packages/gym/core.py", line 333, in close
    return self.env.close()
  File "/home/gong/miniconda3/envs/sacd/lib/python3.9/site-packages/gym/core.py", line 333, in close
    return self.env.close()
  File "/home/gong/miniconda3/envs/sacd/lib/python3.9/site-packages/gym/core.py", line 333, in close
    return self.env.close()
  [Previous line repeated 1 more time]
  File "/home/gong/Projects/sac-discrete-pytorch/smb_env_fct.py", line 180, in close
    env.close()
  File "/home/gong/miniconda3/envs/sacd/lib/python3.9/site-packages/gym/core.py", line 333, in close
    return self.env.close()
  File "/home/gong/miniconda3/envs/sacd/lib/python3.9/site-packages/gym/core.py", line 333, in close
    return self.env.cl